In [1]:
from langchain_aws import ChatBedrock
from bs4 import BeautifulSoup
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

llm = ChatBedrock(
    model_id="us.anthropic.claude-3-haiku-20240307-v1:0",
    region = "us-east-1",    
    model_kwargs=dict(temperature=0)
)



In [56]:
from extractor.core.base_extractor import BaseExtractor
from extractor.params.extraction_params import (
    CDTI_MAIN_DIR, 
    IMPLICITY_WAIT_TIME
)

class CDTIAidExtractor(BaseExtractor):

    def __init__(
        self, 
        instrument,
        support_ambit,
        name,
        url):

        super().__init__()
        self.instrument=instrument
        self.support_ambit=support_ambit
        self.name=name
        self.url=url
        self.persist_data_dir = self.persist_data_dir + "/" + CDTI_MAIN_DIR + "/" + self.get_name_parsed()

        self.implicity_wait_time = IMPLICITY_WAIT_TIME


    def get_name_parsed(self):
        return self.name.replace(" ", "_").lower()

    
    def extract_description(self, url = None):

        url = url if url else self.url
        
        driver = cdti_aid_extractor.setup_driver()
        driver.get(url)
        driver.implicitly_wait(self.implicity_wait_time)

        try:
            aid_title = driver.find_element(By.CLASS_NAME, "block-field-blocknodeayudastitle")
        except: 
            aid_title = None
        try:
            aid_description = driver.find_element(By.CLASS_NAME, "block-field-blocknodeayudasbody")
        except:
            aid_description = None

        if aid_description:
            aid_title_text = aid_title.text
            aid_description_text = aid_description.text  
            aid_description_md = "\n".join([chunk.strip() for chunk in aid_description_text.split("\n")])
            aid_description_md = "# " + aid_title.text + "\n" + aid_description_text

            driver.quit()

            return aid_title_text, aid_description_md

        else:
            driver.quit()
            return None, None


    
    def extract_aid_card(self, url=None):

        url = url if url else self.url

        driver = cdti_aid_extractor.setup_driver()
        driver.get(url)
        driver.implicitly_wait(self.implicity_wait_time)
        try:
            aid_card_element = driver.find_element(By.CLASS_NAME, "card-body")
        except:
            aid_card_element = None

        if aid_card_element:
        
            aid_card_md = ""
            card_fields = aid_card_element.find_elements(By.CLASS_NAME, "ficha-field-wrapper")

            for card_field in card_fields:
                field_title = card_field.find_element(By.CLASS_NAME, "title").text.strip()
                field_text = card_field.find_element(By.CLASS_NAME, "text").text.strip()
                aid_card_md += f"# {field_title}\n{field_text}\n\n"

            driver.quit()

            return aid_card_md

        else:
            driver.quit()
            return None
        

    def extract_aid_doc_url(self, url=None):

        url = url if url else self.url

        driver = cdti_aid_extractor.setup_driver()
        driver.get(url)
        driver.implicitly_wait(self.implicity_wait_time)

        try:
            aid_card_element = driver.find_element(By.CLASS_NAME, "card-body-normativa")
        except:
            aid_card_element = None
            
        if aid_card_element:
            a_aid = aid_card_element.find_element(By.TAG_NAME, "a")
            aid_url = a_aid.get_attribute("href")

            driver.quit()
            return aid_url
        else:
            driver.quit()
            return None


    def extract_aid_subpage(self, url=None):

        url = url if url else self.url

        driver = cdti_aid_extractor.setup_driver()
        driver.get(url)
        driver.implicitly_wait(self.implicity_wait_time)

        try:
            aid_card_element = driver.find_element(By.CLASS_NAME, "view-display-id-block_1")
        except:
            aid_card_element = None

        if aid_card_element:

            try:
                a_element = aid_card_element.find_elements(By.TAG_NAME, "a")
            except:
                a_element = None

            subpage_name = a_element[0].text
            subpage_url = a_element[0].get_attribute("href")

            driver.quit()

            return subpage_name, subpage_url

        else:
            driver.quit()
            return None, None



    # def run_aid_extraction(self):

    #     self.driver.get(self.url)
    #     self.driver.implicitly_wait(5)

    #     self.extract_description()
    #     self.extract_aid_card()

    #     return self



In [85]:
matrix_data = pd.read_csv('data/cdti/matrix_data/cdti_matrix_data.csv')
aid_data = matrix_data.iloc[5]
aid_data

instrument       Ayuda parcialmente reembolsable
support_ambit                         Innovación
name                 Línea Directa de Innovación
url               https://www.cdti.es/node/18458
Name: 5, dtype: object

In [86]:
cdti_aid_extractor = CDTIAidExtractor(
    instrument=aid_data['instrument'],
    support_ambit=aid_data['support_ambit'],
    name=aid_data['name'],
    url=aid_data['url']
)
cdti_aid_extractor.get_name_parsed()

'línea_directa_de_innovación'

In [87]:
# extract description
extraction_data = {}

print(f"extracting aid: {cdti_aid_extractor.name} - {cdti_aid_extractor.instrument}, {cdti_aid_extractor.support_ambit}")
description_title, description_body = cdti_aid_extractor.extract_description()

if description_title:
    print("extracted description:", len(description_body))
    extraction_data['description'] = {
        'description_title': description_title,
        'description_body': description_body
    }

else:
    print("No description found")

# extract card
aid_card = cdti_aid_extractor.extract_aid_card()

if aid_card:
    print("extracted card:", len(aid_card))
    extraction_data['aid_card'] = aid_card
else:
    print("No card found")

# extract doc url
aid_doc_url = cdti_aid_extractor.extract_aid_doc_url()
if aid_doc_url:
    print("extracted aid_doc: ", aid_doc_url)
    extraction_data['aid_doc_url'] = aid_doc_url

else:
    print("No doc url found")
    print("Extracting subpage")

    # extract subpage
    subpage_name, subpage_url = cdti_aid_extractor.extract_aid_subpage()

    if subpage_name:
        print(f"found subpage: {subpage_name}, {subpage_url}")
        extraction_data['subpage'] = {
            "subpage_name": subpage_name,
            "subpage_url": subpage_url
        }

        print("extracting subpage")
        description_title_subpage, description_body_subpage = cdti_aid_extractor.extract_description(url=subpage_url)
        if description_title_subpage:
            print("extracted description:", len(description_body_subpage))
            extraction_data['subpage']['description_title_subpage'] = description_title_subpage
            extraction_data['subpage']['description_body_subpage'] = description_body_subpage

        else:
            print("No description subpage found")

        # extract card
        aid_card_subpage = cdti_aid_extractor.extract_aid_card(url=subpage_url)
        if aid_card_subpage:
            print("extracted card:", len(aid_card_subpage))
            extraction_data['subpage']['aid_card_subpage'] = aid_card_subpage
        else:
            print("No card subpage found")

        # extract doc url
        aid_doc_subpage_url = cdti_aid_extractor.extract_aid_doc_url(url=subpage_url)
        if aid_doc_subpage_url:
            print("extracted aid_doc: ", aid_doc_subpage_url)
            extraction_data['subpage']['aid_doc_subpage_url'] = aid_doc_subpage_url
        else:
            print("No doc subpage url found")


extracting aid: Línea Directa de Innovación - Ayuda parcialmente reembolsable, Innovación
extracted description: 298
extracted card: 3601
extracted aid_doc:  https://www.cdti.es/sites/default/files/2025-03/lic.pdf


In [ ]:
# TODO: crear metodo en clase

# formateo de datos de salida para que el contenido de subpage se meta dentro del main
# si hay description en main y subpage, concatenarlo
# lo que no haya en main y si en subpage, llevarselo ahi
# el resultado tiene que ser:
# {'description': {'description_title': '',
#   'description_body': ''},
#  'aid_card': '',
#  'aid_doc_url': ''}

# persistencia de datos
# crear los ficheros con la info, texto plano en md y pdfs para los docs. buscar forma de
# descargar en el directorio el pdf desde python

extraction_data

{'description': {'description_title': 'Línea Directa de Innovación',
  'description_body': '# Línea Directa de Innovación\nApoyo a proyectos de carácter aplicado, muy cercanos al mercado, con riesgo tecnológico medio/bajo y cortos períodos de recuperación de la inversión, que consigan mejorar la competitividad de la empresa mediante la incorporación de tecnologías emergentes en el sector.'},
 'aid_card': '# Actuación\nLínea Directa de Innovación (LIC)\n\n# Beneficiarios\nEmpresas (proyectos individuales).\n\n# Tipo de convocatoria\nAbierta todo el año.\n\n# Plazo de presentación\nTodo el año.\n\n# Tipo de la ayuda\nAyuda parcialmente reembolsable.\n\n# Características de la ayuda\nAyuda de hasta el 75% del presupuesto aprobado (hasta al 85% si va cofinanciada con FEDER).\nTramo no reembolsable (calculado sobre un máximo del 75% del presupuesto aprobado):\nFondos CDTI: 7%\nFondos Europeos: 10%\nTipo de interés fijo con dos opciones en función del período de amortización elegido:\nAmorti

In [64]:
print(description_body_subpage)

# Actuación conjunta ISCIII-CDTI 2023
El CDTI concederá ayudas parcialmente reembolsables a los proyectos de I+D desarrollados por empresas en cooperación con centros y organismos de investigación en el ámbito de la medicina personalizada y terapias emergentes.
Los proyectos deberán encajar en la misión definida, sobre terapias avanzadas y medicina de precisión enfocadas a:
Envejecimiento
Enfermedades crónicas


In [21]:
print(cdti_aid_extractor.extract_aid_card())

None


In [50]:
# busqueda de url de normativa
print(cdti_aid_extractor.extract_aid_doc_url())

None


In [ ]:
cdti_aid_extractor.extract_aid_subpage()<

('ACTUACIÓN CONJUNTA ISCIII-CDTI 2023',
 'https://www.cdti.es/ayudas/actuacion-conjunta-isciii-cdti-2023')

In [ ]:
# extraccion de subconvocatoria
# view-id-ayudas
# view-display-id-block_1
driver = cdti_aid_extractor.setup_driver()
driver.get(cdti_aid_extractor.url)
driver.implicitly_wait(3)
try:
    aid_card_element = driver.find_element(By.CLASS_NAME, "view-display-id-block_1")
except:
    aid_card_element = None

try:
    a_element = aid_card_element.find_elements(By.TAG_NAME, "a")
except:
    a_element = None

print(a_element[0].text, a_element[0].get_attribute("href"))

ACTUACIÓN CONJUNTA ISCIII-CDTI 2023 https://www.cdti.es/ayudas/actuacion-conjunta-isciii-cdti-2023


In [ ]:


print(a_element[0].get_attribute("href"))

https://www.cdti.es/ayudas/actuacion-conjunta-isciii-cdti-2023


In [ ]:
try:
    a_element = aid_card_element.find_elements(By.TAG_NAME, "a")
except:
    a_element = None

In [16]:
print(cdti_aid_extractor.url)

https://www.cdti.es/node/115


In [28]:
driver = cdti_aid_extractor.driver
url = cdti_aid_extractor.url

driver.get(url)
driver.implicitly_wait(5)
# find matrix block
# div_element = driver.find_element(By.CLASS_NAME, "block-field-blocknodeayudasbody")

In [54]:
# aid description

aid_description = driver.find_element(By.CLASS_NAME, "block-field-blocknodeayudasbody").text
aid_description = "\n".join([chunk.strip() for chunk in aid_description.split("\n")])
print(aid_description)

Los Proyectos de I+D son ayudas a proyectos de I+D desarrollados por empresas y destinados a la creación y mejora significativa de procesos productivos, productos o servicios.
Se presentan en distintas modalidades:
Proyectos de I+D individuales: presentados por una única empresa.
Proyectos de I+D de Cooperación Nacional: presentados por un consorcio de un mínimo de dos y un máximo de seis empresas autónomas. Deberán suscribir un acuerdo privado de colaboración que rija el consorcio.
Proyectos de I+D de Cooperación Tecnológica Internacional: presentados por empresas españolas participantes en programas de cooperación tecnológica internacional gestionados por el CDTI:
Programas multilaterales EUREKA, IBEROEKA y PRIMA.
Programas bilaterales establecidos por CDTI con Instituciones y Agencias de Financiación de otros países en virtud de acuerdos bilaterales.
Programa de proyectos internacionales con certificación y seguimiento unilateral por CDTI.
Proyectos de I+D de Cooperación Tecnológica

In [ ]:
aid_card = {}

aid_card_element = driver.find_element(By.CLASS_NAME, "card-body")
card_fields = aid_card_element.find_elements(By.CLASS_NAME, "ficha-field-wrapper")
for card_field in card_fields:
    field_title = card_field.find_element(By.CLASS_NAME, "title").text.strip()
    field_text = card_field.find_element(By.CLASS_NAME, "text").text.strip()

    aid_card[str(field_title)] = field_text
    # print(field_title, "\n", field_text, "\n\n")
    aid_card += f"# {field_title}\n{field_text}\n\n"

print(aid_card)

# Actuación
Proyectos CDTI de I+D

# Objetivo general de la actuación
Ayudas a proyectos de I+D desarrollados por empresas y destinados a la creación y mejora significativa de procesos productivos, productos o servicios.
Pueden comprender tanto actividades de investigación industrial como de desarrollo experimental.
No existe ninguna restricción en cuanto al sector o a la tecnología a desarrollar.

# Beneficiarios
Empresas.

# Tipo de convocatoria
Convocatoria abierta todo el año.

# Plazo de presentación
Todo el año.

# Tipo de la ayuda
Ayuda parcialmente reembolsable.

# Características de la ayuda
Tipo de interés fijo: Euribor a 1 año.
Ayuda de hasta el 85% del presupuesto aprobado.
La empresa deberá aportar, al menos, el 15% del presupuesto del proyecto con recursos propios.
Anticipo de hasta el 50% de la ayuda con límite de 300.000 euros, sin exigencia de garantías adicionales.
Tramo no reembolsable de entre el 10% y el 33% de la ayuda.
Devolución: 10 o 15 años incluyendo una care

In [ ]:
# pdf convocatoria cuidaooo
# a veces viene como ficha del instrumento y a veces como listado de convocatorias, que hay que coger la primera
aid_call_document = driver.find_element(By.CLASS_NAME, "view-id-ayudas")
aid_call_document.text


'Ficha resumen de la ayuda\nActuación\nProyectos CDTI de I+D\nObjetivo general de la actuación\nAyudas a proyectos de I+D desarrollados por empresas y destinados a la creación y mejora significativa de procesos productivos, productos o servicios.\nPueden comprender tanto actividades de investigación industrial como de desarrollo experimental.\nNo existe ninguna restricción en cuanto al sector o a la tecnología a desarrollar.\nBeneficiarios\nEmpresas.\n  Tipo de convocatoria\nConvocatoria abierta todo el año.\nPlazo de presentación\nTodo el año.\nTipo de la ayuda\nAyuda parcialmente reembolsable. \nCaracterísticas de la ayuda\nTipo de interés fijo: Euribor a 1 año.\nAyuda de hasta el 85% del presupuesto aprobado.\nLa empresa deberá aportar, al menos, el 15% del presupuesto del proyecto con recursos propios.\nAnticipo de hasta el 50% de la ayuda con límite de 300.000 euros, sin exigencia de garantías adicionales.\nTramo no reembolsable de entre el 10% y el 33% de la ayuda.\nDevolución: 1

In [ ]:
'''
formato de enlaces

* normal: como i+d: en el enlace principal viene:
    * body
    * ficha
    * enlace a pdf: ficha del instrumento

* caso 1: Actuación conjunta ISCIII-CDTI: 
    * hay un body
    hay una lista de enlaces de convocatorias presentes y pasadas. Hay que coger la primera
    * en ese enlace:
        * otro body mas pequeño
        * ficha
        * enlace a pdf (ficha del instrumento)
* caso 2: Programa Tecnológico de Automoción Sostenible (PTAS):
    * no hay body, aunque podria haberlo
    * ficha
    * enlace a pdf: documentacion convocatorias
* caso 3: CDTI-Era-Net:
    * body
    * ficha
    * lista de enlaces a convocatorias. coger la primera.
        * otro body
        enlace a pdf: texto de la convocatoria
'''

In [ ]:
'''
mas o menos generico:

texto descripcion: 
block-field-blocknodeayudastitle
block-field-blocknodeayudasbody

ficha: card-body

enlace documento pdf: card-body-normativa

cuando hay varias opciones de convocatorias:
view-id-ayudas
view-display-id-block_1

'''

In [ ]:
'''
flujo del scrapper

entra en url desde matrix
busca title y body de descripcion. 
pillar title para nombre documento
Si existe el body, genera un md con el contenido
busca ficha del documento. Si exsite, genera un md con el contenido
busca enlace a pdf de convocatoria (card-body-normativa). 
if existe:
    lo guarda en pdf
else:
    busca enlace de subpaginas: 
        view-id-ayudas
        view-display-id-block_1
    Sacar las url y pillar la primera que es la mas actual.
    a esa url quizas haya que añadirle https://www.cdti.es/

    en esa pagina repetimos:
    busca title y body de descripcion. 
    pillar title para nombre documento
    Si existe el body, genera un md con el contenido
    busca ficha del documento. Si exsite, genera un md con el contenido

'''

In [ ]:
necesito un codigo en python con selenium que haga lo siguiente:

rellena un diccionario con los siguiente:
a partir de una url busca:
comprueba si existen los elementos de clase block-field-blocknodeayudastitle yblock-field-blocknodeayudasbody
En caso de existir, extrae el texto de ambos, los concatena y lo guarda en el diccionario con el nombre "aid_description_1"
Busca el elemento de clase card-body, y si existe, extrae todo el texto que contiene concatenado, de todos los elementos de clase ficha-field-wrapper. Los concatena y lo guarda en el diccionario con el nombre "aid_card_1"
ahora busca el elemento card-body-normativa
si existe, almacena la url en el diccionario con el nombre "aid_call_document_url"
si no existe:
comprueba si hay mas enlaces a subdocumentos. busca el elemento view-id-ayudas y extrae la primera url que aparece.

Con selenium, en esa url, repite el proceso de busqueda de los elementos block-field-blocknodeayudastitle, block-field-blocknodeayudasbody, card-body y card-body-normativa
En caso de existir cada uno, los almacena en json de la misma forma, pero empleando el sufijo _2

In [47]:
field_text

'Estas ayudas se ajustarán a lo establecido en el Reglamento (UE) n.º 651/2014, de la Comisión, de 17 de junio de 2014, por el que se declaran determinadas categorías de ayuda compatibles con el mercado interior en aplicación de los artículos 107 y 108 del Tratado (Reglamento general de exención por categorías).'

In [37]:
card_fields = aid_card.find_elements(By.CLASS_NAME, "ficha-field-wrapper")
for card_field in card_fields:
    field_title = card_field.find_element(By.CLASS_NAME, "title").text.strip()
    field_text = card_field.find_element(By.CLASS_NAME, "text").text.strip()
    print(field_title, "\n", field_text, "\n\n")


Actuación 
 Proyectos CDTI de I+D 


Objetivo general de la actuación 
 Ayudas a proyectos de I+D desarrollados por empresas y destinados a la creación y mejora significativa de procesos productivos, productos o servicios.
Pueden comprender tanto actividades de investigación industrial como de desarrollo experimental.
No existe ninguna restricción en cuanto al sector o a la tecnología a desarrollar. 


Beneficiarios 
 Empresas. 


Tipo de convocatoria 
 Convocatoria abierta todo el año. 


Plazo de presentación 
 Todo el año. 


Tipo de la ayuda 
 Ayuda parcialmente reembolsable. 


Características de la ayuda 
 Tipo de interés fijo: Euribor a 1 año.
Ayuda de hasta el 85% del presupuesto aprobado.
La empresa deberá aportar, al menos, el 15% del presupuesto del proyecto con recursos propios.
Anticipo de hasta el 50% de la ayuda con límite de 300.000 euros, sin exigencia de garantías adicionales.
Tramo no reembolsable de entre el 10% y el 33% de la ayuda.
Devolución: 10 o 15 años incluye

In [35]:
card_field = card_fields[0]
card_field.find_element(By.CLASS_NAME, "title").text
card_field.find_element(By.CLASS_NAME, "text").text

'Proyectos CDTI de I+D'

In [ ]:
url = "https://ejemplo.com"  # Reemplaza con la URL que necesitas
driver.get(url)

try:
    # Encontrar el div por clase
    div_element = driver.find_element(By.CLASS_NAME, "block-field-blocknodeayudasbody")
    
    # Obtener todos los elementos dentro del div
    inner_elements = div_element.find_elements(By.XPATH, ".//*")
    
    # Extraer y mostrar los textos
    for element in inner_elements:
        text = element.text.strip()
        if text:
            print(text)
finally:
    driver.quit()